In [1]:
import json
from transformers import BertTokenizerFast
import os
from tqdm import tqdm

In [6]:
# Initialize the tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

def preprocess_dataset(input_path, output_dir, is_train=False):
    with open(input_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    processed_data = []
    chunk_count = 0  # Track the number of chunks

    for i, entry in enumerate(tqdm(data, desc="Processing data")):
        context = entry['context']
        target_dialogue = entry['target_dialogue']
        speaker_positions = entry['speaker_position']

        # Tokenize and get offset mappings
        encoding = tokenizer(
            context,
            truncation=True,
            max_length=512,
            padding='max_length',
            return_offsets_mapping=True
        )

        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        offsets = encoding['offset_mapping']
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        # Align target dialogue with token indices
        target_start_char, target_end_char = target_dialogue
        target_token_indices = [
            j for j, (start, end) in enumerate(offsets)
            if start >= target_start_char and end <= target_end_char
        ]
        target_indices = [target_token_indices[0], target_token_indices[-1]] if target_token_indices else [0, 0]

        # Align speaker positions with token indices
        speaker_indices = [
            [
                j for j, (start, end) in enumerate(offsets)
                if (start >= sp_start and end <= sp_end)
            ]
            for sp_start, sp_end in speaker_positions
        ]
        valid_speaker_indices = [
            [idx[0], idx[-1]] for idx in speaker_indices if idx and (idx[-1] - idx[0]) < 20
        ]
        if not valid_speaker_indices:
            print("empty valid_speaker_indices")
            continue
        target_tokens = tokens[target_indices[0]:target_indices[1] + 1]
        speaker_tokens = [
            tokens[start:end + 1] for (start, end) in valid_speaker_indices
        ]
        # Store the preprocessed entry
        processed_data.append({
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'target_indices': target_indices,
            'target_tokens': target_tokens,
            'speaker_indices': valid_speaker_indices,
            'speaker_tokens': speaker_tokens
        })
        # If is_train is True, save every 2048 entries as a chunk
        if is_train and len(processed_data) >= 2048:
            chunk_path = os.path.join(output_dir, f"chunk_{chunk_count}.json")
            with open(chunk_path, 'w', encoding='utf-8') as f:
                json.dump(processed_data, f, ensure_ascii=False, indent=4)
            print(f"Saved chunk {chunk_count} with 2048 entries to {chunk_path}")
            processed_data = []  # Reset for the next chunk
            chunk_count += 1
    # Save the preprocessed dataset
    if processed_data:
        if is_train:
            chunk_path = os.path.join(output_dir, f"chunk_{chunk_count}.json")
            with open(chunk_path, 'w', encoding='utf-8') as f:
                json.dump(processed_data, f, ensure_ascii=False, indent=4)
            print(f"Saved final chunk {chunk_count} to {chunk_path}")
        else:
            output_path = os.path.join(output_dir, os.path.basename(input_path).replace('processed_', ''))
            with open(output_path, 'w', encoding='utf-8') as f:
                json.dump(processed_data, f, ensure_ascii=False, indent=4)
            print(f"Saved processed data to {output_path}")

In [9]:

input_path = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\LiteraryTextsDataset\dataset\processed_train.json"
output_path = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\train"
preprocess_dataset(input_path, output_path, is_train=True)

Processing data:   2%|▏         | 445/21226 [00:00<00:31, 663.23it/s]

empty valid_speaker_indices
empty valid_speaker_indices


Processing data:   7%|▋         | 1444/21226 [00:02<00:27, 732.24it/s]

empty valid_speaker_indices


Processing data:   8%|▊         | 1676/21226 [00:02<00:27, 702.69it/s]

...

empty valid_speaker_indices


Processing data: 100%|█████████▉| 21154/21226 [00:45<00:00, 534.08it/s]

empty valid_speaker_indices
empty valid_speaker_indices


Processing data: 100%|██████████| 21226/21226 [00:45<00:00, 467.83it/s]


empty valid_speaker_indices
Saved final chunk 10 to C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\train\chunk_10.json


In [10]:
input_path = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\LiteraryTextsDataset\dataset\processed_val.json"
output_path = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001"
preprocess_dataset(input_path, output_path)

Processing data:  13%|█▎        | 228/1793 [00:00<00:02, 766.45it/s]

empty valid_speaker_indices
empty valid_speaker_indices
empty valid_speaker_indices


Processing data:  22%|██▏       | 388/1793 [00:00<00:01, 782.78it/s]

empty valid_speaker_indices


Processing data:  31%|███       | 555/1793 [00:00<00:01, 805.60it/s]

...

empty valid_speaker_indices
empty valid_speaker_indices
empty valid_speaker_indices


Processing data:  80%|████████  | 1437/1793 [00:01<00:00, 763.27it/s]

empty valid_speaker_indices


Processing data: 100%|██████████| 1793/1793 [00:02<00:00, 779.46it/s]


Saved processed data to C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\val.json


In [11]:
input_path = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\LiteraryTextsDataset\dataset\processed_test.json"
output_path = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001"
preprocess_dataset(input_path, output_path)

Processing data:  17%|█▋        | 614/3669 [00:00<00:03, 785.42it/s]

empty valid_speaker_indices
empty valid_speaker_indices


Processing data:  28%|██▊       | 1014/3669 [00:01<00:03, 782.94it/s]

empty valid_speaker_indices
empty valid_speaker_indices


Processing data:  36%|███▋      | 1331/3669 [00:01<00:03, 740.06it/s]

...

Processing data:  95%|█████████▌| 3503/3669 [00:04<00:00, 777.64it/s]

empty valid_speaker_indices
empty valid_speaker_indices


Processing data: 100%|██████████| 3669/3669 [00:04<00:00, 741.64it/s]

empty valid_speaker_indices
empty valid_speaker_indices


Saved processed data to C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\test.json


In [3]:
import os
import json

def count_entries_in_json(file_path):
    """
    Counts the number of entries in a JSON file.
    
    Args:
        file_path (str): Path to the JSON file.

    Returns:
        int: Number of entries in the JSON file.
    """
    with open(file_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
        return len(data)

def generate_dataset_report(train_dir, test_file, val_file):
    """
    Generates a report summarizing the number of entries in the dataset.

    Args:
        train_dir (str): Path to the directory containing training JSON files.
        test_file (str): Path to the test JSON file.
        val_file (str): Path to the validation JSON file.
    """
    total_train_entries = 0
    train_files = [f for f in os.listdir(train_dir) if f.endswith('.json')]

    # Count total entries in train directory
    for train_file in train_files:
        train_file_path = os.path.join(train_dir, train_file)
        entries = count_entries_in_json(train_file_path)
        total_train_entries += entries

    # Count entries in test and validation files
    test_entries = count_entries_in_json(test_file)
    val_entries = count_entries_in_json(val_file)

    # Generate and print the report
    print("===== Dataset Report =====")
    print(f"Total Training Entries: {total_train_entries}")
    print(f"Total Validation Entries: {val_entries}")
    print(f"Total Test Entries: {test_entries}")
    print("==========================")

# Example usage
train_dir = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\train"
val_file = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\val.json"
test_file = r"C:\Users\Lenovo\OneDrive\NUS\CS-24fall\project\AudiobookGeneration_cs5647\Dataset_SID_001\test.json"

generate_dataset_report(train_dir, test_file, val_file)


===== Dataset Report =====
Total Training Entries: 21111
Total Validation Entries: 1779
Total Test Entries: 3641
